In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy.engine import URL, create_engine, Inspector

# Use your own server connection here
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=DAIYAAN;"
            "Database=AIFMRM_ERS;"
            "Trusted_Connection=yes;")

cnxn_url = URL.create("mssql+pyodbc", query={"odbc_connect": cnxn_str})

engine = create_engine(cnxn_url)


# initialise connection via context manager           
with engine.connect() as cnxn:
        tables_df = pd.read_sql('SELECT [name] AS [table_name] FROM sys.tables', cnxn)
        table_name_list = tables_df.table_name
        select_template = 'SELECT * FROM {table_name}'
        # Dictionary of table names and their respective SQL queries
        frames_dict = {}
        for tname in table_name_list:
                query = select_template.format(table_name = tname)
                frames_dict[tname] = pd.read_sql(query, cnxn)

Function 1

In [ ]:
def GetICsAndWeights(rDate,IndexCode,dbo_tbl_Index_Constituents):
    #Store tbl_Index_Constituents from frames_dict
    tbl_Index_Constituents = frames_dict[dbo_tbl_Index_Constituents]

    #rDate will be supplied by the user: consisting of year and Quarter 
    rDate = rDate
    rDate = pd.to_datetime(rDate, format = "%Y-%m-%d")
    rDate_Quarter = rDate.quarter
    rDate_Year = rDate.year

    #search tbl_Index_Constituents Date column and find Quarter and Year for each date in column
    Dates_Col = tbl_Index_Constituents["Date"]
    Dates_Col = pd.arrays.DatetimeArray(Dates_Col)
    Dates_Col_Quarter = Dates_Col.quarter
    Dates_Col_Year = Dates_Col.year

    #Filter tbl_Index_Constituents using supplied quarter and year data from rData
    tbl_Index_Constituents_Date = tbl_Index_Constituents.loc[(Dates_Col_Quarter == rDate_Quarter) & (Dates_Col_Year == rDate_Year),]


    #IndexCode is provided by user: "ALSI", "FLED", "LRGC", "MIDC", "SMLC", "TOPI", "RESI", "FINI", "INDI", "PCAP", "SAPY" or "ALTI"
    IndexCode = IndexCode #provided as input by the user. "ALSI" for testing purposes

    #function to identify The index column that must be searched
    def Index_Col_Identifier(argument):
        match argument:
            case "FLED":
                return "ALSI New"
            case  "LRGG"|"MIDC"|"SMILC":
                return "Index New"
            case default:
                return argument+" New"
    
    IndexCode_Col = Index_Col_Identifier(IndexCode) #Obtain column name to search relevant rows

    #Filter tbl_Index_Constituents_Date using supplied IndexCode in the column identified from Index_Col_Identifier function
    tbl_Index_Constituents_final = tbl_Index_Constituents_Date[tbl_Index_Constituents_Date[IndexCode_Col] == IndexCode]

    #Generate results table with Shares and corresponding share weights
    Alpha = pd.DataFrame(tbl_Index_Constituents_final.loc[:,"Alpha"])
    Gross_Market_Capitalisation = np.array(tbl_Index_Constituents_final.loc[:,"Gross Market Capitalisation"])
    Weigths = pd.DataFrame(Gross_Market_Capitalisation/np.sum(Gross_Market_Capitalisation))
    Results = pd.concat([Alpha.reset_index(drop=True), Weigths.reset_index(drop=True)],axis=1)
    Results.columns = ['Alpha','Weights']

    return Results


rDate = '2019-6-05'
IndexCode = "ALSI"
dbo_tbl_Index_Constituents = "tbl_Index_Constituents"
Output1 = GetICsAndWeights(rDate,IndexCode,dbo_tbl_Index_Constituents)

Function 2

In [ ]:
def GetBetasMktAndSpecVols(rDate,ICs,dbo_tbl_BA_Beta_Output,mktIndexCode):

    #Store tbl_BA_Beta_Output from frames_dict
    tbl_BA_Beta_Output = frames_dict[dbo_tbl_BA_Beta_Output]

    #rDate will be supplied by the user: consisting of year and Quarter 
    rDate = rDate
    rDate = pd.to_datetime(rDate, format = "%Y-%m-%d")
    rDate_Quarter = rDate.quarter
    rDate_Year = rDate.year

    #search tbl_Index_Constituents Date column and find Quarter and Year for each date in column
    Dates_Col = tbl_BA_Beta_Output["Date"]
    Dates_Col = pd.arrays.DatetimeArray(Dates_Col)
    Dates_Col_Quarter = Dates_Col.quarter
    Dates_Col_Year = Dates_Col.year

    #Filter tbl_BA_Beta_Output using supplied quarter and year data from rData
    tbl_BA_Beta_Output_Date = tbl_BA_Beta_Output.loc[(Dates_Col_Quarter == rDate_Quarter) & (Dates_Col_Year == rDate_Year),]
    #return tbl_BA_Beta_Output_Date
    #Market index code provided by the user which could be "J203", "J200", "J250", "J257" or "J258"
    mktIndexCode = mktIndexCode

    #Filter tbl_BA_Beta_Output_Dates using provided mktIndexCode
    tbl_BA_Beta_Output_mktIndex = tbl_BA_Beta_Output_Date.loc[tbl_BA_Beta_Output_Date["Index"] == mktIndexCode]
    #return tbl_BA_Beta_Output_mktIndex

    #list of IndexCodes obtain from  the Alpha column in the Output of Function 1.
    ICs = ICs
    tbl_BA_Beta_Output_IC = tbl_BA_Beta_Output_mktIndex.loc[tbl_BA_Beta_Output_mktIndex["Instrument"].isin(ICs)]

    #Generate results table with Shares and corresponding share weights
    Betas = tbl_BA_Beta_Output_IC.loc[:,"Beta"]
    specVols = tbl_BA_Beta_Output_IC.loc[:,"Unique Risk"]
    mktVol = tbl_BA_Beta_Output_IC.loc[:,"Total Risk"]
    Results = pd.concat([Betas.reset_index(drop=True), specVols.reset_index(drop=True),mktVol.reset_index(drop=True)],axis=1)
    Results.columns = ['Betas','specVols','mktVol']

    return Results
    #return tbl_BA_Beta_Output_IC

ICs = Output1["Alpha"]
dbo_tbl_BA_Beta_Output = "tbl_BA_Beta_Output"
mktIndexCode = "J203"
Output2 = GetBetasMktAndSpecVols(rDate,ICs,dbo_tbl_BA_Beta_Output,mktIndexCode)

Function 3

In [ ]:
def CalcStats(weights,betas,specVols,mktVol):

    weights = np.transpose(np.matrix(Output1["Weights"]))
    betas = np.transpose(np.matrix(Output2["Betas"]))
    specVols = np.transpose(np.matrix(Output2["specVols"]))
    mktVol = np.array(Output2["mktVol"])
    mktVol = np.average(mktVol) #Assuming the mktVol is the average. NEED TO CONFIRM

    pfBeta = np.matmul(np.transpose(weights),betas)

    sysCov = np.matmul(betas,np.transpose(betas))*(mktVol**2)

    pfSysVol = np.transpose(weights)@betas@np.transpose(betas)@(weights)*(mktVol**2)

    specCov = np.diagflat(specVols)@np.diagflat(specVols)

    pfSpecVol = np.transpose(weights)@(specCov)@weights

    totCov = sysCov + specCov

    pfVol = pfSysVol + pfSpecVol